In [6]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR

from tqdm.auto import tqdm
import nltk
import string
from collections import Counter
from typing import List

In [7]:
# Простой искусственный словарь
vocab = ['<pad>', '<bos>', '<eos>', '<unk>', 'hello', 'love', 'you', 'more', 'then', 'live', 'my']
word2ind = {word: i for i, word in enumerate(vocab)}
ind2word = {i: word for word, i in word2ind.items()}

# Пример датасета, представляющего последовательности слов в виде индексов
sentences = [
    [word2ind['love'], word2ind['you'], word2ind['more'],  word2ind['then'], word2ind['live']],
    [word2ind['love'], word2ind['live']],
    [word2ind['love'], word2ind['you']],
    [word2ind['hello'], word2ind['my'], word2ind['love']],
    [word2ind['love'], word2ind['you'], word2ind['more']],
]
print ('sentences', sentences)

# Создаем Dataset
class SimpleDataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __getitem__(self, idx):
        return self.data[idx]
    def __len__(self):
        return len(self.data)

# Функция паддинга для создания батчей
def collate_fn_with_padding(input_batch, pad_id=word2ind['<pad>']):
    seq_lens = [len(x) for x in input_batch]
    max_seq_len = max(seq_lens)
    new_batch = []
    for sequence in input_batch:
        padded_sequence = sequence + [pad_id] * (max_seq_len - len(sequence))
        new_batch.append(padded_sequence)
    sequences = torch.LongTensor(new_batch)
    return {
        'input_ids': sequences[:, :-1],  # Входные данные
        'target_ids': sequences[:, 1:]   # Целевые данные
    }
# Создаем DataLoader для train, eval, test наборов
train_dataset = SimpleDataset(sentences)
eval_dataset = SimpleDataset(sentences[:2])  # возьмем половину для оценки
test_dataset = SimpleDataset(sentences[2:])  # другая половина для тестирования

train_dataloader = DataLoader(train_dataset, collate_fn=collate_fn_with_padding, batch_size=2)
eval_dataloader = DataLoader(eval_dataset, collate_fn=collate_fn_with_padding, batch_size=2)
test_dataloader = DataLoader(test_dataset, collate_fn=collate_fn_with_padding, batch_size=2)

for i, batch in enumerate(train_dataloader):
    print(f"в train Batch {i + 1}:")
    print("в train Input и Target IDs состоит из элементов:", batch)
for i, batch in enumerate(eval_dataloader):
    print(f"в eval Batch {i + 1}:")
    print("в eval Input IDs состоит из элементов:", batch)
for i, batch in enumerate(test_dataloader):
    print(f"в test Batch {i + 1}:")
    print("в test Input IDs состоит из элементов:", batch)

# Модель
class LanguageModel(nn.Module):
    def __init__(self, hidden_dim: int, vocab_size: int):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.lstm  = nn.LSTM(hidden_dim, hidden_dim, num_layers=1, batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.projection = nn.Linear(hidden_dim, vocab_size)
        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(p=0.2)
    def forward(self, input_batch: torch.Tensor) -> torch.Tensor:
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        output, _ = self.lstm(embeddings)  # [batch_size, seq_len, hidden_dim]
        output = self.dropout(self.linear(self.non_lin(output)))  # [batch_size, seq_len, hidden_dim]
        projection = self.projection(self.non_lin(output))  # [batch_size, seq_len, vocab_size]
        return projection

# Инициализация модели, функции потерь, оптимизатора, планировщика
hidden_dim = 16
model = LanguageModel(hidden_dim, len(vocab)).to('cpu')  # для демонстрации используем CPU
criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
optimizer = torch.optim.Adam(model.parameters())
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)

# Функция для вывода содержимого батча
def print_batch_info(batch, batch_num):
    print(f"\n--- Batch {batch_num} ---")
    print(f"Input IDs (input_batch):\n{batch['input_ids']}")
    print(f"Target IDs (target_batch):\n{batch['target_ids']}")

# Модифицированная функция тренировки с подробным выводом
def train_model_verbose(model, criterion, optimizer, scheduler, train_dataloader, eval_dataloader, epochs=5):
    perplexities = []
    losses = []
    for epoch in tqdm(range(epochs), desc="epoch"):
        print(f"\nEpoch {epoch+1}/{epochs}")
        model.train()  # Установка режима обучения
        epoch_losses = []

        for batch_num, train_batch in enumerate(tqdm(train_dataloader, desc="train"), 1):
            # Выводим информацию о текущем батче
            print_batch_info(train_batch, batch_num)
            # Получение предсказаний от модели (логитов)
            #Размер логита равен размеру словаря vocab

            output = model(train_batch["input_ids"]).flatten(start_dim=0, end_dim=1)

            print(f"\nLogits (predicted probabilities) for Batch {batch_num}:\n{output}")
            # Вычисление потерь
            loss = criterion(output, train_batch["target_ids"].flatten())
            print(f"Loss for Batch {batch_num}: {loss.item()}")
            # Обратный проход (backward) для вычисления градиентов
            loss.backward()
            # Шаг оптимизатора
            optimizer.step()
            optimizer.zero_grad()  # Обнуляем градиенты
            # Сохраняем потери для данной эпохи
            epoch_losses.append(loss.item())
        # Шаг планировщика скорости обучения
        scheduler.step()
        # Вычисляем средние потери за эпоху и сохраняем их
        average_loss = sum(epoch_losses) / len(epoch_losses)
        losses.append(average_loss)
        print(f"Average loss for Epoch {epoch+1}: {average_loss}")

        # Оценка на валидационном наборе
        perplexity = evaluate(model, criterion, eval_dataloader)
        perplexities.append(perplexity)
        print(f"Perplexity after Epoch {epoch+1}: {perplexity}")

    return perplexities, losses

# Функция оценки остается прежней
def evaluate(model, criterion, dataloader) -> float:
    model.eval()
    perplexity = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="validation"):
            logits = model(batch['input_ids']).flatten(start_dim=0, end_dim=1)
            loss = criterion(logits, batch['target_ids'].flatten())
            perplexity.append(torch.exp(loss).item())
    return sum(perplexity) / len(perplexity)

# Запуск модифицированной функции с детальным выводом
perplexities, losses = train_model_verbose(model, criterion, optimizer, scheduler, train_dataloader, eval_dataloader, epochs=2)

sentences [[5, 6, 7, 8, 9], [5, 9], [5, 6], [4, 10, 5], [5, 6, 7]]
в train Batch 1:
в train Input и Target IDs состоит из элементов: {'input_ids': tensor([[5, 6, 7, 8],
        [5, 9, 0, 0]]), 'target_ids': tensor([[6, 7, 8, 9],
        [9, 0, 0, 0]])}
в train Batch 2:
в train Input и Target IDs состоит из элементов: {'input_ids': tensor([[ 5,  6],
        [ 4, 10]]), 'target_ids': tensor([[ 6,  0],
        [10,  5]])}
в train Batch 3:
в train Input и Target IDs состоит из элементов: {'input_ids': tensor([[5, 6]]), 'target_ids': tensor([[6, 7]])}
в eval Batch 1:
в eval Input IDs состоит из элементов: {'input_ids': tensor([[5, 6, 7, 8],
        [5, 9, 0, 0]]), 'target_ids': tensor([[6, 7, 8, 9],
        [9, 0, 0, 0]])}
в test Batch 1:
в test Input IDs состоит из элементов: {'input_ids': tensor([[ 5,  6],
        [ 4, 10]]), 'target_ids': tensor([[ 6,  0],
        [10,  5]])}
в test Batch 2:
в test Input IDs состоит из элементов: {'input_ids': tensor([[5, 6]]), 'target_ids': tensor([[6, 

epoch:   0%|          | 0/2 [00:00<?, ?it/s]


Epoch 1/2


train:   0%|          | 0/3 [00:00<?, ?it/s]


--- Batch 1 ---
Input IDs (input_batch):
tensor([[5, 6, 7, 8],
        [5, 9, 0, 0]])
Target IDs (target_batch):
tensor([[6, 7, 8, 9],
        [9, 0, 0, 0]])

Logits (predicted probabilities) for Batch 1:
tensor([[ 0.0025, -0.2177,  0.2278, -0.0397,  0.2736, -0.2622, -0.1385,  0.1948,
          0.1828,  0.1053,  0.1636],
        [-0.0473, -0.0991,  0.2604,  0.0079,  0.1836, -0.1412, -0.1601,  0.2215,
          0.2242, -0.0170,  0.0589],
        [-0.0231, -0.0591,  0.1918, -0.0008,  0.1874, -0.1387, -0.1494,  0.2385,
          0.2181,  0.0242,  0.1283],
        [-0.0248, -0.0816,  0.2618, -0.0770,  0.1799, -0.1429, -0.1027,  0.3719,
          0.2127,  0.0129,  0.0859],
        [ 0.0266, -0.1158,  0.2493, -0.0952,  0.3076, -0.2155, -0.1193,  0.1931,
          0.2367,  0.1267,  0.2090],
        [-0.0148, -0.1624,  0.1774, -0.0885,  0.0834, -0.1674, -0.0847,  0.2007,
          0.0917,  0.0808,  0.2290],
        [-0.0092,  0.0012,  0.2608, -0.0422,  0.2283, -0.0688, -0.1287,  0.2585,
     

...

Perplexity after Epoch 1: 10.658402442932129

Epoch 2/2


train:   0%|          | 0/3 [00:00<?, ?it/s]


--- Batch 1 ---
Input IDs (input_batch):
tensor([[5, 6, 7, 8],
        [5, 9, 0, 0]])
Target IDs (target_batch):
tensor([[6, 7, 8, 9],
        [9, 0, 0, 0]])

Logits (predicted probabilities) for Batch 1:
tensor([[-0.1052, -0.0558,  0.1619,  0.0118,  0.1907, -0.1506, -0.1428,  0.1845,
          0.1978,  0.1263,  0.1787],
        [-0.0280, -0.2053,  0.2108, -0.0923,  0.2059, -0.2175, -0.1030,  0.2178,
          0.2101,  0.0179,  0.1399],
        [-0.0475, -0.1132,  0.1614, -0.0280,  0.0547, -0.1173, -0.1144,  0.3044,
          0.1370, -0.0112,  0.0768],
        [-0.0663, -0.1767,  0.1660, -0.1065,  0.1966, -0.1658, -0.0765,  0.2710,
          0.1710,  0.0037,  0.1074],
        [-0.1031, -0.0929,  0.1474, -0.0398,  0.2548, -0.1436, -0.1210,  0.2075,
          0.2378,  0.0436,  0.1029],
        [-0.0368, -0.0916,  0.1890, -0.1314,  0.1839, -0.1790, -0.1099,  0.1785,
          0.1853,  0.0726,  0.2459],
        [-0.0194,  0.0033,  0.2249, -0.0494,  0.2141, -0.0834, -0.0678,  0.3461,
     

validation:   0%|          | 0/1 [00:00<?, ?it/s]

Perplexity after Epoch 2: 10.558945655822754


```
Размер логита равен размеру словаря vocab ['<pad>', '<bos>', '<eos>', '<unk>', 'hello', 'love', 'you', 'more', 'then', 'live', 'my']

Для 1 батча:
    После flatten: [8, 11] (2×4=8 предсказаний, каждое размером со словарь)

Количество строк = batch_size × sequence_length

Рассмотрим конкретный пример:
tensor([[5, 6, 7, 8],   # "love you more then"
        [5, 9, 0, 0]])  # "love live <pad> <pad>"

Target IDs (target_batch):
tensor([[6, 7, 8, 9],   # "you more then live"
        [9, 0, 0, 0]])  # "live <pad> <pad> <pad>"

Для первой последовательности:
После "love" должно быть "you"
После "love you" должно быть "more"
После "love you more" должно быть "then"

Для второй последовательности:
После "love" должно быть "live"
После "love live" должно быть "<pad>"
После "love live <pad>" должно быть "<pad>"

Языковая модель учится предсказывать следующее слово с учетом всего предыдущего контекста.
в коде прописано,что нужно брать именно [5, 6] ("love you") для 7 ("more"), а не только соотвествующий [6] ("you") для 7 ("more")
Это определяется архитектурой LSTM в модели:

При получении последовательности [5, 6] для предсказания 7
Сначала обрабатывается 5 ("love") и обновляется внутреннее состояние LSTM
Затем, при обработке 6 ("you"), LSTM использует свое предыдущее состояние, содержащее информацию о "love"


```